<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [3]:
import pandas as pd

# Read in the Amazon data into a dataframe
df_amzn = pd.read_csv("data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

In [6]:
# df_amzn.head()

In [7]:
# Imports
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danoand/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
# Gensim and tools imports
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [11]:
# Set up stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [15]:
df_amzn.columns

Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
      dtype='object')

In [38]:
# Create a working dataframe 
df_amzn_wrk = df_amzn.copy()

In [39]:
# Tokenize the review text documents

# Import spacy module
import spacy
# Create a spacy object
nlp = spacy.load('en_core_web_lg')

# tknz_strings tokenizes a string passed to the function
def tknz_strings(strng):
    # Generate tokens - including removal of punctuation
    return gensim.utils.simple_preprocess(strng, deacc=True)

# rmv_stopwords removes stopwords from a list of tokens
def rmv_stopwords(lst_tkns):
    return [wrd for wrd in lst_tkns if wrd not in stop_words]

# mke_lemmanade lemmatizes the passed tokens
def mke_lemmanade(lst_tkns, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    ret_tkns = []
    
    # Ingest the token list via spacy
    tmp_doc = nlp(" ".join(lst_tkns))
    
    # From the spacy output, grab the lemmatized token objects
    for nlp_tkn in tmp_doc:
        # Ignore the token if it's not a desired part of speech
        if nlp_tkn.pos_ not in allowed_postags:
            continue
            
        # Include this lemmatized token
        ret_tkns.append(nlp_tkn.lemma_)
        
    # Post iteration: return our list of lemmatized tokens
    return ret_tkns      

In [40]:
# Generate initial tokens from the review text
df_amzn_wrk['upd_tokens'] = df_amzn_wrk['reviews.text'].apply(tknz_strings)

In [41]:
# Remove stopwords from each document token list
df_amzn_wrk['upd_tokens'] = df_amzn_wrk['upd_tokens'].apply(rmv_stopwords)

In [42]:
# Lemmatize each document token list
df_amzn_wrk['upd_tokens'] = df_amzn_wrk['upd_tokens'].apply(mke_lemmanade)

In [44]:
df_amzn_wrk['upd_tokens'].sample(10)

8068     [cheap, battery, poor, performance, put, remot...
6732                            [hold, charge, top, brand]
9360                  [great, battery, life, great, price]
13294    [wife, enjoy, much, buy, second, first, reader...
10190                               [good, battery, price]
25884    [detail, item, little, sparse, thought, would,...
16740    [pick, couple, hour, ago, screen, good, tablet...
2377                                        [great, price]
21014              [good, tablet, regulation, little, low]
25052    [really, like, idea, tablet, affordable, niece...
Name: upd_tokens, dtype: object

In [45]:
# Create an normalized word (token) to integer id (LDA Dictionary)

# Create a list of review tokens represented in our work dataframe
upd_tokens_list = df_amzn_wrk['upd_tokens'].tolist()

# Generate the dictionary
id2word = corpora.Dictionary(upd_tokens_list)

In [52]:
# Map a term id to term frequency corpus
corpus = [id2word.doc2bow(txt) for txt in upd_tokens_list]

In [55]:
# Build an LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=42,
                                           update_every=1,
                                           chunksize=50,
                                           passes=8,
                                           alpha='auto',
                                           per_word_topics=True)

In [56]:
# Visualize topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.323020 -0.243050       1        1  34.469040
0      0.151699  0.295353       2        1  20.820961
3     -0.176936 -0.031216       3        1  18.930799
4     -0.222554 -0.180523       4        1  13.220356
2     -0.075228  0.159437       5        1  12.558844, topic_info=             Term          Freq         Total Category  logprob  loglift
2961       tablet  11454.000000  11454.000000  Default  30.0000  30.0000
27          great   8694.000000   8694.000000  Default  29.0000  29.0000
59           good   6293.000000   6293.000000  Default  28.0000  28.0000
93           love   6768.000000   6768.000000  Default  27.0000  27.0000
97            use   8374.000000   8374.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
485    especially    326.263916    327.019348   Topic5  -4.8215   2.0724
474   inexpensive    313.733246    314.498413   Topic5  -4.8607   2.0723
458        family    294.638428    295.395203   Topic5  -4.9235   2.0722
459        friend    263.430328    264.185455   Topic5  -5.0354   2.0719
1062         unit    259.256073    260.016907   Topic5  -5.0514   2.0718

[184 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
974       2  0.997998  absolutely
1477      4  0.999221      access
472       4  0.998311     account
791       2  0.998505          ad
963       4  0.999521         add
...     ...       ...         ...
26        1  0.999674        well
629       2  0.998989        wife
12        1  0.999874        work
116       3  0.999805       would
105       1  0.999681        year

[169 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 5, 3])

## Notes

* The choice of 5 topics looks fairly decent
* With one or two exceptions, the top 30 terms in each topic are almost exclusive to their assigned topic

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling